This notebook is about determining which $n$-grams we should make into features. More specifically, for which $x$ should we add a feature of the form "contains a harmonically equivalent $n$-gram to $x$"?
In theory, we could do this for any harmonically unique n-gram for $n=1,2,3,4,...$ but that would be infinitely many different features, which is obviously impossible.

So how to pick which ones are good? 

* Based on domain knowledge, sequences longer than, say, $10$ are definitely not going to be useful for distinguishing genre. First of all, I can't imagine there are any $10$-grams which actually occur in more than $1\%$ of the songs, but also musically speaking, a chord progression of length $10$ is usually longer than an entire "musical idea" or "phrase." Even $10$ is pushing it, really based on this idea we shouldn't expect anything longer than $6$ to be very useful.
* Even if we wanted to look at all the harmonically unique $n$-grams for $1 \le n \le 6$, that would be far too many. There are $44$ harmonically unique $1$-grams in the training set, $5903$ unqiue $2$-grams. To determine the list of unique $2$-grams takes a three minute computation, but I project to determine the counts for unique 3-grams, the computation would take around $24$ hours.
* Because of the above, we need to heavily filter our choice of n-grams. The first easy criterion is to only consider $n$-grams which appear in some sample size threshold of the data set, maybe $1-10\%$. The table below gives concrete numbers for the number of unique $1$-grams and $2$-grams after restricting to higher and higher relative sample size requirements. Based on the table, I think a threshold cutoff of around $1\%$ is a good sweet spot.

| $n$      | $n$-grams | $1\%$ | $2\%$ | $5\%$ |$10\%$ |
| :-: | :-: | :-: | :-: | :-: | :-: |
| 1 | 44 | 22 | 20 | 15 | 10
| 2 | 5903 | 237 | 173 | 105 | 62 |

* Because compiling a full list of unique $3$-grams seems to be computationally taxing (and for $4, 5, 6$ definitely infeasible), we are probably going to need to simplify something. Even though it takes really long to compile the list of unique $3$-grams, it is very fast to compile the complete list of raw $3$-grams (less than $30$ seconds). This computation doesn't scale up much with $n$, in fact even up to $n=9$ it is still well under a minute.

In [2]:
import pandas as pd
from collections import Counter
import numpy as np
import json

data_folder_path = '../../data/'

In [3]:
# read in the database
df = pd.read_csv(data_folder_path + 'clean_test.csv', low_memory=False)
num_songs = len(df.index)

In [4]:
print("Number of songs:",num_songs)
display(df.head(5))

Number of songs: 255606


,chords,simplified_chords,decade,main_genre,spotify_song_id
0,<intro_1> G A Fsmin Bmin G A Fsmin Bmin <verse...,"G,A,Fsmin,Bmin,G,A,Fsmin,Bmin,G,A,Fsmin,Bmin,G...",2010.0,pop,7vpGKEUPrA4UEsS4o4W1tP
1,C F G C F G F Dmin G C F Dmin G C F G C F G F ...,"C,F,G,C,F,G,F,Dmin,G,C,F,Dmin,G,C,F,G,C,F,G,F,...",2000.0,alternative,7MTpNQUBKyyymbS3gPuqwQ
2,C F C G Amin G F C F C G Amin G F C G C F C G ...,"C,F,C,G,Amin,G,F,C,F,C,G,Amin,G,F,C,G,C,F,C,G,...",2000.0,alternative,6jIIMhcBPRTrkTWh3PXIc7
3,Amin G Gmin B Amin G Gmin B Amin G Gmin B Amin...,"Amin,G,Gmin,B,Amin,G,Gmin,B,Amin,G,Gmin,B,Amin...",2010.0,pop,2zAfQdoOeYujy7QIgDUq9p
4,<verse_1> D Dmaj7 G/D A/D D Dmaj7 G/D A/D <cho...,"D,Dmaj7,G,A,D,Dmaj7,G,A,G,D,Emin,D,A,G,D,Emin,...",2010.0,metal,40rChMoUd1VXb4TKgTuTSP


In [5]:
# specify limitations on n, and sample size threshold
n_max = 10
n_range = list(range(1, n_max+1))
relative_sample_size_threshold = 0.01
sample_size_threshold = relative_sample_size_threshold * num_songs

In [6]:
print("Sample size threshold:",int(sample_size_threshold))

Sample size threshold: 2556


In [7]:
# load the counter json files
with open(data_folder_path + 'harmonically_unique_1_gram_counts.json') as file:
    unique_1_gram_counter = Counter(json.load(file))
with open(data_folder_path + 'harmonically_unique_2_gram_counts.json') as file:
    unique_2_gram_counter = Counter(json.load(file))

In [8]:
unique_1_gram_counter.most_common(10)

[('G', 12777669),
 ('Fsmin', 4627067),
 ('Gs7', 594128),
 ('Emin7', 548196),
 ('Cno3d', 353019),
 ('Dmaj7', 277398),
 ('Dsus4', 222221),
 ('Cadd9', 109279),
 ('A9', 34895),
 ('Eminadd13', 31283)]

In [9]:
unique_2_gram_counter.most_common(10)

[('F,C', 2945027),
 ('C,F', 2359941),
 ('G,A', 1230986),
 ('G,F', 1063415),
 ('G,Amin', 995946),
 ('Bmin,G', 861782),
 ('Amin,G', 852222),
 ('A,Fsmin', 822318),
 ('Amin,C', 700118),
 ('Eb,Gmin', 597621)]

In [10]:
def get_filtered_counter(my_counter, threshold):
    return Counter({ key : value for key, value in my_counter.items() if value >= threshold})

In [11]:
# Eliminate any entries which don't meet the sample size threshold
filtered_1_gram_counter = get_filtered_counter(unique_1_gram_counter, sample_size_threshold)
filtered_2_gram_counter = get_filtered_counter(unique_2_gram_counter, sample_size_threshold)

In [12]:
print(len(filtered_1_gram_counter))
print(len(filtered_2_gram_counter))

22
237


In [14]:
def get_raw_n_gram_counts(chord_column, n):
    # compile a dictionary of counts
    results = Counter()
    for song in chord_column:
        song_as_list = song.split(',')
        song_n_grams = [','.join(song_as_list[i:i+n]) for i in range(len(song_as_list) - n + 1)]
        for ng in song_n_grams:
            results[ng] += 1
    return results

In [20]:
raw_counters = [get_raw_n_gram_counts(df['simplified_chords'], n) for n in n_range]

In [21]:
for n in n_range:
    index = n-1
    print("n=",n)
    print("Number of raw n-grams:",len(raw_counters[index]))
    print(raw_counters[index].most_common(10))
    print()

n= 1
Number of raw n-grams: 690
[('G', 2625626), ('C', 2191805), ('D', 2003664), ('A', 1533155), ('F', 1294437), ('Amin', 1107254), ('E', 1052746), ('Emin', 996675), ('Bmin', 575690), ('B', 548336)]

n= 2
Number of raw n-grams: 42574
[('C,G', 706503), ('G,D', 591016), ('G,C', 554507), ('D,G', 480181), ('F,C', 445031), ('D,A', 396728), ('A,D', 337546), ('C,F', 315429), ('A,E', 309189), ('G,Amin', 275016)]

n= 3
Number of raw n-grams: 298213
[('G,C,G', 200871), ('C,G,C', 171186), ('C,G,D', 170030), ('D,G,D', 157471), ('F,C,G', 155828), ('G,D,G', 142521), ('C,F,C', 133546), ('G,D,A', 121522), ('C,G,Amin', 117324), ('A,D,A', 115067)]

n= 4
Number of raw n-grams: 888910
[('C,G,C,G', 95210), ('G,C,G,C', 90135), ('G,D,G,D', 71195), ('D,G,D,G', 69352), ('F,C,F,C', 64615), ('C,F,C,F', 61359), ('D,A,D,A', 53756), ('F,C,G,Amin', 53013), ('C,G,Amin,F', 52877), ('A,E,A,E', 52701)]

n= 5
Number of raw n-grams: 1752485
[('G,C,G,C,G', 63053), ('C,G,C,G,C', 58360), ('D,G,D,G,D', 46406), ('C,F,C,F,C', 4

In [22]:
# a generic method for iterating through a counter of n-grams and aggregating equivalent n-grams
def uniquify_n_grams(n_gram_counter, n, progress_updates = False):
    results = Counter()
    processed = set()
    if progress_updates:
        t0 = time.time()
        i = 0
        percent_complete = 0
        num_raw = len(n_gram_counter)
        print("Finding counts of " + str(n) + "-grams up to harmonic equivalence.")
        print("There are " + str(num_raw) + " raw " + str(n) + "-grams to process.")
        progress_interval = int(num_raw /100)
    for ng1 in n_gram_counter:
        if progress_updates:
            i += 1
            if i % progress_interval == 0 and i !=0:
                percent_complete += 1
                print("Processing " + str(n) + "-gram number " + str(i) + ".")
                print("\tTime spent so far:",time.time() - t0)
                print("\tComputation is " + str(percent_complete) + "% complete.")
        if ng1 in processed:
            continue
        total = n_gram_counter[ng1]
        for ng2 in n_gram_counter:
            if (ng2 not in processed) and ng1 != ng2:
                if compare_n_grams(ng1, ng2)[0]:
                    total += n_gram_counter[ng2]
                    processed.add(ng2)
        results[ng1] = total
        processed.add(ng1)
    return results